In [ ]:
from transformers import pipeline



In [ ]:
generator = pipeline('text-generation', model = 'gpt2')
generator("I HATE MY JOB", max_length = 30, num_return_sequences=3)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},

## Idea 1
Train text generator on Amazon bad reviews


In [ ]:
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00")

In [ ]:
import pandas as pd

data_df = pd.DataFrame.from_dict(dataset['train'])
data_df.head()

In [ ]:
bad_reviews = data_df.loc[data_df["star_rating"] == 1]
len(bad_reviews)

In [ ]:
bad_reviews["review_body"].head()

# Cleaning Data

In [ ]:
# a little sentiment analysis
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
sentiment_pipeline = pipeline("text-classification", model=model_name)

In [ ]:
sentiment_pipeline.__dict__

In [ ]:
review = bad_reviews["review_body"].values[0]
sentiment_pipeline(review)

In [ ]:
cleaner_data = None

for index, bad_review in bad_reviews.iterrows():
    sentiment = sentiment_pipeline(bad_review["review_body"])[0]
    if (sentiment["label"] == "POSITIVE"):
        cleaner_data = bad_reviews.drop(index=index, axis = 0)
    elif sentiment["score"] < 0.8:
        cleaner_data = bad_reviews.drop(index=index, axis = 0)


In [ ]:
len(cleaner_data )

# Retraining on top of transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
bad_reviews_array = bad_reviews["review_body"].values.tolist()
bad_reviews_array[0:2]

In [ ]:
tokenized_bad_reviews = tokenizer(bad_reviews_array, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
type(tokenized_bad_reviews)

In [ ]:
# 
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bad_reviews,
    data_collator=lambda data: {"input_ids": torch.stack([f[0] for f in data]), "attention_mask": torch.stack([f[1] for f in data]), "labels": torch.stack([f[0] for f in data])},
)

trainer.train()